In [3]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [4]:
os.environ['GROQ_API_KEY']=os.getenv('GROQ_API')

In [5]:
from groq import Groq
import base64
from IPython.display import Image,display,Markdown
import pandas as pd

In [6]:
MODEL = 'meta-llama/llama-4-scout-17b-16e-instruct' #meta-llama/llama-4-maverick-17b-128e-instruct
client = Groq()

In [7]:
# Define the system prompt with detailed instructions
system_prompt = """
Convert the menu image to a structured excel sheet format following the provided template and instructions.
This assistant converts restaurant or cafe menu data into a structured Excel sheet that adheres to a specific template.
The template includes categories, subcategories, item names, prices, descriptions, and more, ensuring data consistency.
This assistant helps users fill out each row correctly, following the detailed instructions provided.

Overview:
- Each row in the Excel spreadsheet represents a unique item, categorized under a category or subcategory.
- Category and subcategory names are repeated for items within the same subcategory.
- Certain columns are left blank when not applicable, such as subcategory details for items directly under a category.
- Item details, including names, prices, and descriptions, must be unique for each entry.
- Uploaded menu content will be appended to the existing menu without deleting any current entries.

Columns Guide:

Column Name                    | Description                               | Accepted Values           | Example
-------------------------------|-------------------------------------------|---------------------------|-----------------------
CategoryTitlePt (Column A)      | Category names in Portuguese              | Text, 256 characters max  | Bebidas
CategoryTitleEn (Column B) (Optional) | English translations of category titles | Text, 256 characters max  | Beverages
SubcategoryTitlePt (Column C) (Optional) | Subcategory titles in Portuguese | Text, 256 characters max or blank | Sucos
SubcategoryTitleEn (Column D) (Optional) | English translations of subcategory titles | Text, 256 characters max or blank | Juices
ItemNamePt (Column E)           | Item names in Portuguese                  | Text, 256 characters max  | Água Mineral
ItemNameEn (Column F) (Optional) | English translations of item names | Text, 256 characters max or blank | Mineral Water
ItemPrice (Column G)          | Price of each item without currency symbol  | Text                      | 2.50 or 2,50
Calories (Column H) (Optional) | Caloric content of each item              | Numeric                   | 150
PortionSize (Column I)        | Portion size for each item in units        | Text                      | 500ml, 1, 2-3
Availability (Column J) (Optional) | Current availability of the item     | Numeric: 1 for Yes, 0 for No | 1
ItemDescriptionPt (Column K) (Optional) | Detailed description in Portuguese | Text, 500 characters max  | Contains essential minerals
ItemDescriptionEn (Column L) (Optional) | Detailed description in English | Text, 500 characters max  | Contains essential minerals

Notes:
- Ensure all data entered follows the specified formats to maintain database integrity.
- Review the data for accuracy and consistency before submitting the Excel sheet.
"""


In [8]:
import pandas as pd

In [19]:
directory = "Regatta"

In [20]:
IMAGE_DIR = directory

def encode_image(image_path):
    with open(image_path,'rb') as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')


images_files = sorted([f for f in os.listdir(IMAGE_DIR) if f.lower().endswith(('.png','.jpg','.jpeg'))])
images_files

['Regatta_page_1.jpg',
 'Regatta_page_2.jpg',
 'Regatta_page_3.jpg',
 'Regatta_page_4.jpg',
 'Regatta_page_5.jpg',
 'Regatta_page_6.jpg',
 'Regatta_page_7.jpg']

In [21]:
new_excel_filename = input('Enter the new excel file name (without extension): ')
EXCEL_PATH = os.path.join(directory,f"{new_excel_filename}.xlsx")


df = pd.DataFrame(columns=['CategoryTitlePt', 'CategoryTitleEn', 'SubcategoryTitlePt', 'SubcategoryTitleEn',
                           'ItemNamePt', 'ItemNameEn', 'ItemPrice', 'Calories', 'PortionSize', 'Availability',
                           'ItemDescriptionPt', 'ItemDescriptionEn'])

                           
for image in images_files:
    image_path = os.path.join(IMAGE_DIR,image)
    image_data = encode_image(image_path)

    headers_added = False

    response = client.chat.completions.create(
        model = MODEL,
        messages = [
            {'role':'system','content':system_prompt},
            {'role':'user','content':[
                {'type':'text',
                'text':'convert this menu image to a structured excel sheet format.'},
                {'type':'image_url',
                'image_url':{'url':f"data:image/png;base64,{image_data}"}}
            ]}
        ],
        temperature=0
    )

    for row in response.choices[0].message.content.split('\n'):
        if row.startswith('|') and not row.startswith('|-'): # Ensure that the data is a row and not a header format
            columns = [col.strip() for col in row.split('|')[1:-1]]
            if len(columns) == len(df.columns):
                if 'CategoryTitlePt' in columns:
                    headers_added = True
                    continue
                if headers_added and 'CategoryTitlePt' in columns:
                    continue # skip the row
                new_row = pd.Series(columns, index=df.columns)
                df = pd.concat([df, pd.DataFrame([new_row])], ignore_index=True)
            else:
                print(f"Skipping row { row}")


Skipping row | ÁGUAS & REFRIGERANTES | WATER & SOFT DRINKS |  |  | Iced Tea / Sumol Laranja / Coca-Cola / Coca-Cola Zero | Iced Tea / Sumol Laranja / Coca-Cola / Coca-Cola Zero |  |  |  |  |  |


In [22]:
df

,CategoryTitlePt,CategoryTitleEn,SubcategoryTitlePt,SubcategoryTitleEn,ItemNamePt,ItemNameEn,ItemPrice,Calories,PortionSize,Availability,ItemDescriptionPt,ItemDescriptionEn
0,---,---,---,---,---,---,---,---,---,---,---,---
1,ENTRADAS,STARTERS,,,Azeitonas / Olives,Olives,1.60,,,,,
2,ENTRADAS,STARTERS,,,Cesto de Pão / Bread Basket,Bread Basket,1.80,,,,,
3,ENTRADAS,STARTERS,,,Creme de Legumes,Cream of Vegetables,3.00,,,,,
4,PARA PARTILHAR,TO SHARE,,,Nachos & Guacamole,,7.50,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...
86,Digestivos & Licores,Digestives & Liqueurs,,,Moscate Favaíos,Moscate Favaíos,2.00,,,1,,
87,Digestivos & Licores,Digestives & Liqueurs,,,Licor Beirão,Licor Beirão,3.00,,,1,,
88,Digestivos & Licores,Digestives & Liqueurs,,,Vinho do Porto,Vinho do Porto,3.00,,,1,,
89,Digestivos & Licores,Digestives & Liqueurs,,,Whisky,Whisky,5.00,,,1,,


In [15]:
df.drop(0,axis=0,inplace=True)

In [16]:
df= df.reset_index()

In [17]:
df.drop(columns=['index'],inplace=True)
df

,CategoryTitlePt,CategoryTitleEn,SubcategoryTitlePt,SubcategoryTitleEn,ItemNamePt,ItemNameEn,ItemPrice,Calories,PortionSize,Availability,ItemDescriptionPt,ItemDescriptionEn
0,ENTRADAS,STARTERS,,,Azeitonas / Olives,Olives,1.60,,,,,
1,ENTRADAS,STARTERS,,,Cesto de Pão / Bread Basket,Bread Basket,1.80,,,,,
2,ENTRADAS,STARTERS,,,Creme de Legumes,Cream of Vegetables,3.00,,,,,
3,PARA PARTILHAR,TO SHARE,,,Nachos & Guacamole,,7.50,,,,,
4,PARA PARTILHAR,TO SHARE,,,REGATTA Tuna,,12.90,,,,Atum fresco marinado / Marinated Tuna,
...,...,...,...,...,...,...,...,...,...,...,...,...
95,Digestivos & Licores,Digestives & Liqueurs,,,Moscate Favaíos,Moscate Favaíos,2.00,,,1,,
96,Digestivos & Licores,Digestives & Liqueurs,,,Licor Beirão,Licor Beirão,3.00,,,1,,
97,Digestivos & Licores,Digestives & Liqueurs,,,Vinho do Porto,Vinho do Porto,3.00,,,1,,
98,Digestivos & Licores,Digestives & Liqueurs,,,Whisky,Whisky,5.00,,,1,,


In [24]:
df.to_excel(EXCEL_PATH,index=False)
print(f"Excel file saved at: {EXCEL_PATH}")

Excel file saved at: Regatta\regatta.xlsx


In [49]:
#display(Markdown(response.choices[0].message.content))